In [2]:
from depsurf import DBGSYM_PATH

from elftools.elf.elffile import ELFFile

f = open(DBGSYM_PATH / "6.5.0-27-generic", "rb")
elffile = ELFFile(f)
dwarfinfo = elffile.get_dwarf_info()

In [10]:
from collections import defaultdict

from depsurf import reload_depsurf

reload_depsurf()

from depsurf import (
    DIEHandler,
    Traverser,
    disable_dwarf_cache,
    get_name,
    SubroutineEntry,
)
from elftools.dwarf.die import DIE


subroutine_info = {}


def filter_subprogram(die: DIE):
    assert die.tag == "DW_TAG_subprogram"

    # ignore declaration-only subprograms
    decl = die.attributes.get("DW_AT_declaration")
    if decl is not None and decl.value:
        return False

    # ignore inlined subprograms as they will be accounted at the call site
    aa = die.attributes.get("DW_AT_abstract_origin")
    if aa is not None:
        return False

    return True


def get_entry(die: DIE, traverser: Traverser):
    name = get_name(die)
    if name is None:
        return None, None

    info_dict = subroutine_info.get(name)
    if info_dict is None:
        info_dict = defaultdict(SubroutineEntry)
        subroutine_info[name] = info_dict

    extern = die.attributes.get("DW_AT_external")
    if extern is not None:
        assert extern.value
        info = info_dict[name]
        info.external = True
    else:
        location = traverser.get_die_location(die)
        info = info_dict[location]
        info.external = False
    return info, name


def handle_subprogram(die: DIE, traverser: Traverser):
    get_entry(die, traverser)


def resolve_abstract_origin(die: DIE):
    if "DW_AT_abstract_origin" not in die.attributes:
        return die

    return resolve_abstract_origin(die.get_DIE_from_attribute("DW_AT_abstract_origin"))


def handle_call_impl(die: DIE, traverser: Traverser, is_inline: bool):
    fn_die = resolve_abstract_origin(die)
    entry, name = get_entry(fn_die, traverser)
    if entry is None:
        return

    if name == "vfs_read" and is_inline:
        print("found vfs_read")

    caller = traverser.curr_subprogram
    assert caller is not None
    if is_inline:
        entry.caller_inline.append(caller)
    else:
        entry.caller_func.append(caller)


def handle_call_site(die: DIE, traverser: Traverser):
    handle_call_impl(die, traverser, is_inline=False)


def handle_inlined_subroutine(die: DIE, traverser: Traverser):
    handle_call_impl(die, traverser, is_inline=True)


handler_map = {
    "DW_TAG_compile_unit": DIEHandler(show=True, rec=True),
    "DW_TAG_subprogram": DIEHandler(
        show=True,
        rec=True,
        filter_fn=filter_subprogram,
        handle_fn=handle_subprogram,
    ),
    "DW_TAG_inlined_subroutine": DIEHandler(
        show=True,
        rec=True,
        handle_fn=handle_inlined_subroutine,
    ),
    "DW_TAG_GNU_call_site": DIEHandler(
        show=True,
        rec=False,
        handle_fn=handle_call_site,
    ),
    "DW_TAG_lexical_block": DIEHandler(show=False, rec=True),
}


Traverser.enable_show = True
cu = next(cu for cu in dwarfinfo.iter_CUs() if cu.get_top_DIE().get_full_path().endswith("main.c"))
Traverser(cu, handler_map).traverse()

compile_unit: /build/linux-hwe-6.5-jkqeMi/linux-hwe-6.5-6.5.0/init/main.c
  subprogram: console_on_rootfs
    inlined_subroutine: IS_ERR
  subprogram: kernel_init
    inlined_subroutine: exit_boot_config
    inlined_subroutine: mark_readonly
      inlined_subroutine: rodata_test
    inlined_subroutine: try_to_run_init_process
    inlined_subroutine: try_to_run_init_process
    inlined_subroutine: try_to_run_init_process
    inlined_subroutine: try_to_run_init_process
  subprogram: free_initmem
    inlined_subroutine: free_initmem_default
  subprogram: mark_readonly (Not declared inline but inlined by the compiler)
  subprogram: set_debug_rodata
  subprogram: arch_parse_debug_rodata (Declared inline and inlined by the compiler)
  subprogram: kernel_init_freeable
    inlined_subroutine: get_current
    inlined_subroutine: task_pid
    inlined_subroutine: get_pid
      inlined_subroutine: refcount_inc
        inlined_subroutine: __refcount_inc
          inlined_subroutine: __refcount_add


In [ ]:
from depsurf import disable_dwarf_cache

disable_dwarf_cache()

Traverser.enable_show = False
for cu in dwarfinfo.iter_CUs():
    Traverser(cu, handler_map).traverse()

In [5]:
import pickle

with open("subroutine_info.pkl", "wb") as f:
    pickle.dump(subroutine_info, f)